In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv').drop_duplicates()
df.head()

In [ ]:
df.info()

In [ ]:
df[df.notnull() == True].shape

In [ ]:
blind_prediction = round(len(df[df.output == 1])/len(df), 2)
blind_prediction

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.subplots(figsize=(10,10))
sns.heatmap(df.corr(), square=True, annot=True, fmt='.2f')

In [ ]:
df.hist(figsize=(16,8))

In [ ]:
numeric_data = [0, 3, 4, 7, 9, 11]
df.iloc[:, numeric_data].plot(subplots=True, figsize = (10, 20))

In [ ]:
for i in range(len(numeric_data)):
    sns.boxplot(data = df.iloc[:, numeric_data[i]], orient='h')
    plt.title(df.columns[[numeric_data[i]]][0])
    plt.show()

In [ ]:
df.describe()

In [ ]:
df.chol.sort_values(ascending=False)

In [ ]:
df.drop([85], inplace=True)
sns.boxplot(data = df.chol, orient='h')

In [ ]:
df.head()

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
from sklearn.preprocessing import scale
X = scale(np.array(df.drop('output', axis = 1), dtype = 'float'), with_std = True, with_mean = True)
y = np.array(df.output)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()
knn_grid = {'n_neighbors': np.array(np.linspace(1, 100, 10), dtype='int')}

gs = GridSearchCV(knn, knn_grid, cv=5, n_jobs = -1)

gs.fit(X, y)

In [ ]:
def grid_plot(x, y, x_label, title, y_label='cross_val'):
    plt.figure(figsize=(12, 6))
    plt.grid(True)
    plt.plot(x, y, 'ro-')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
print(gs.best_params_, gs.best_score_)
grid_plot(knn_grid['n_neighbors'], gs.cv_results_['mean_test_score'], 'n_neighbors', 'KNeighborsClassifier')

In [ ]:
knn_grid = {'n_neighbors': np.array(np.linspace(1, 40, 40), dtype='int')}
gs = GridSearchCV(knn, knn_grid, cv=5, n_jobs = -1)
gs.fit(X, y)
print(gs.best_params_, gs.best_score_)
grid_plot(knn_grid['n_neighbors'], gs.cv_results_['mean_test_score'], 'n_neighbors', 'KNeighborsClassifier')

In [ ]:
knn_grid = {'n_neighbors': np.array(np.linspace(40, 60, 20), dtype='int')}
gs = GridSearchCV(knn, knn_grid, cv=5, n_jobs = -1)
gs.fit(X, y)
print(gs.best_params_, gs.best_score_)
grid_plot(knn_grid['n_neighbors'], gs.cv_results_['mean_test_score'], 'n_neighbors', 'KNeighborsClassifier')

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 7)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

clf_knn = knn.fit(X_train, y_train)
y_knn = clf_knn.predict(X_test)

In [ ]:
y_test

In [ ]:
y_knn

In [ ]:
from sklearn import metrics
print ('knn =', metrics.accuracy_score(y_test, y_knn))
print('blind_prediction =', blind_prediction)

In [ ]:
import matplotlib
fig = plt.figure(figsize=(8,8))
nn_mtx = metrics.confusion_matrix(y_test, y_knn)
print(nn_mtx)
font = {'weight' : 'bold', 'size'   :22}

matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 
sns.heatmap(nn_mtx, annot=True, fmt="d", 
            xticklabels=set(df.output), 
            yticklabels=set(df.output))
plt.ylabel("Real value")
plt.xlabel("Predicted value")

In [ ]:
print('precision =', round(metrics.precision_score(y_test, y_knn), 5))
print('recall =', round(metrics.recall_score(y_test, y_knn), 5))
print('fi =', round(metrics.f1_score(y_test, y_knn), 5))